In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#!pip install -q keras

In [2]:
!pip install -q pydrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 61 kB 504 kB/s 
     |████████████████████████████████| 3.3 MB 45.2 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
project_path='/content/drive/My Drive/Machine Learning/UBC/CPSC_503/SemEval22_T7/'

In [ ]:
#cd $project_path

In [ ]:
ls

In [6]:
import numpy as np
import pandas as pd

In [ ]:
df_data = pd.read_csv(project_path+'Datasets/ClarificationTask_DevData_Oct22a.tsv',sep='\t')
df_labels = pd.read_csv(project_path+'Datasets/ClarificationTask_DevLabels_Oct22a.tsv',sep='\t',names=['Id','Labels'])
df_scores = pd.read_csv(project_path+'Datasets/ClarificationTask_DevScores_Oct22a.tsv',sep='\t',names=['Id','Scores'])

In [ ]:
df_data.head()

,Id,Resolved pattern,Article title,Section header,Previous context,Sentence,Follow-up context,Filler1,Filler2,Filler3,Filler4,Filler5
0,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear
1,1,METONYMIC REFERENCE,How to Deal With Being Single and Feeling Lonely,Focusing on Yourself,(...) 3. Go on a trip. (...) 4. Get active an...,"If you are lonely, take up a new sport or ____...","This is great for your body, as well as your s...",freedom,circle,field,activity,form
2,2,FUSED HEAD,How to Cope With Anxiety and Depression,Seeking Professional Help,(...) 4. Allow medication time to work. If yo...,Most ______ take four to eight weeks to show a...,NaN,disorders,men,medications,people,procedures
3,3,FUSED HEAD,How to Be an Otaku (Anime Geek),Steps,1. Pick and watch some anime of your interest...,"If you want to have the edge over others, then...",2. Try watchingthe anime in the original Japan...,families,humans,men,people,females
4,4,METONYMIC REFERENCE,How to Uninstall Netflix on Samsung Smart TV,Steps,1. Press the Home button.,The home button is the button that resembles t...,This will open the Smart Hub.,video,interior,outline,movement,construction


In [ ]:
df_labels.head(11)

,Id,Labels
0,0_1,NEUTRAL
1,0_2,IMPLAUSIBLE
2,0_3,PLAUSIBLE
3,0_4,PLAUSIBLE
4,0_5,IMPLAUSIBLE
5,1_1,IMPLAUSIBLE
6,1_2,IMPLAUSIBLE
7,1_3,NEUTRAL
8,1_4,NEUTRAL
9,1_5,PLAUSIBLE


In [ ]:
df_new_columns = df_data.columns.tolist()
df_new_columns.extend(['label_1','label_2','label_3','label_4','label_5','score_1','score_2','score_3','score_4','score_5'])
df_data.columns[0]


'Id'

In [ ]:
df_temp=df_scores[df_scores['Id']=='1_4']
type(df_temp['Id'].tolist()[0])

str

In [ ]:
df_new ={}
for c in df_new_columns:
  df_new[c]=[]

for i in range(len(df_data)):
  for c in df_data.columns:
    df_new[c].append(df_data[c][i])
  
  id = df_data['Id'][i]
  for j in range(5):
    df_temp_labels = df_labels[df_labels['Id']==str(id)+'_'+str(j+1)]
    df_new['label_'+str(j+1)].append(str(df_temp_labels['Labels'].tolist()[0])) ############################

    df_temp_scores = df_scores[df_scores['Id']==str(id)+'_'+str(j+1)]
    df_new['score_'+str(j+1)].append(str(df_temp_scores['Scores'].tolist()[0])) ############################


df_new = pd.DataFrame(df_new)
df_new.to_csv(project_path+'Datasets/Clarification_Merge_Test_1.csv',index=False)

print(len(df_new),len(df_data))
df_new.head()

In [ ]:
df_1 = pd.read_csv(project_path+'Datasets/Clarification_Merge_Test_1.csv')

In [ ]:
print(df_1['Sentence'][0])

2. Get surgery for ______ at 3 to 6 months old. 


In [ ]:
print(df_1.columns)

Index(['Id', 'Resolved pattern', 'Article title', 'Section header',
       'Previous context', 'Sentence', 'Follow-up context', 'Filler1',
       'Filler2', 'Filler3', 'Filler4', 'Filler5', 'label_1', 'label_2',
       'label_3', 'label_4', 'label_5', 'score_1', 'score_2', 'score_3',
       'score_4', 'score_5'],
      dtype='object')


In [ ]:
df_1.head()

,Id,Resolved pattern,Article title,Section header,Previous context,Sentence,Follow-up context,Filler1,Filler2,Filler3,Filler4,Filler5,label_1,label_2,label_3,label_4,label_5,score_1,score_2,score_3,score_4,score_5
0,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear,NEUTRAL,IMPLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,3.0,1.5,5.0,5.0,1.0
1,1,METONYMIC REFERENCE,How to Deal With Being Single and Feeling Lonely,Focusing on Yourself,(...) 3. Go on a trip. (...) 4. Get active an...,"If you are lonely, take up a new sport or ____...","This is great for your body, as well as your s...",freedom,circle,field,activity,form,IMPLAUSIBLE,IMPLAUSIBLE,NEUTRAL,NEUTRAL,PLAUSIBLE,1.0,1.0,3.0,3.0,5.0
2,2,FUSED HEAD,How to Cope With Anxiety and Depression,Seeking Professional Help,(...) 4. Allow medication time to work. If yo...,Most ______ take four to eight weeks to show a...,NaN,disorders,men,medications,people,procedures,NEUTRAL,PLAUSIBLE,PLAUSIBLE,PLAUSIBLE,NEUTRAL,3.5,4.5,5.0,4.5,3.5
3,3,FUSED HEAD,How to Be an Otaku (Anime Geek),Steps,1. Pick and watch some anime of your interest...,"If you want to have the edge over others, then...",2. Try watchingthe anime in the original Japan...,families,humans,men,people,females,IMPLAUSIBLE,NEUTRAL,NEUTRAL,PLAUSIBLE,NEUTRAL,2.0,3.5,3.5,4.0,3.0
4,4,METONYMIC REFERENCE,How to Uninstall Netflix on Samsung Smart TV,Steps,1. Press the Home button.,The home button is the button that resembles t...,This will open the Smart Hub.,video,interior,outline,movement,construction,IMPLAUSIBLE,IMPLAUSIBLE,PLAUSIBLE,NEUTRAL,PLAUSIBLE,1.5,2.5,5.0,3.0,4.0


In [ ]:
df_single = {}
df_1_cols = df_1.columns.tolist()

for c in df_1_cols:
  df_single[c] = []

df_single['Filled_Sentence'] = []
df_single['Filler'] = []
df_single['Label'] = []
df_single['Score'] = []


for i in range(len(df_1['Id'])):
  for j in range(5):
    for c in df_1_cols:
      df_single[c].append(df_1[c][i])

    sent = df_1["Sentence"][i]
    filler = str(df_1['Filler'+str(j+1)][i])
    sent = sent.replace("______", filler)
    df_single['Filled_Sentence'].append(sent)

    df_single['Filler'].append(filler)

    df_single['Label'].append(df_1['label_'+str(j+1)][i])

    df_single['Score'].append(df_1['score_'+str(j+1)][i])
  

df_single = pd.DataFrame(df_single)
df_single.to_csv(project_path+'Datasets/Clarification_Merge_Single_Filler_Test_1.csv',index=False)

print(len(df_1),len(df_single))
df_single.head()

500 2500


,Id,Resolved pattern,Article title,Section header,Previous context,Sentence,Follow-up context,Filler1,Filler2,Filler3,Filler4,Filler5,label_1,label_2,label_3,label_4,label_5,score_1,score_2,score_3,score_4,score_5,Filled_Sentence,Filler,Label,Score
0,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear,NEUTRAL,IMPLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,3.0,1.5,5.0,5.0,1.0,2. Get surgery for the split at 3 to 6 months ...,the split,NEUTRAL,3.0
1,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear,NEUTRAL,IMPLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,3.0,1.5,5.0,5.0,1.0,2. Get surgery for your chin at 3 to 6 months ...,your chin,IMPLAUSIBLE,1.5
2,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear,NEUTRAL,IMPLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,3.0,1.5,5.0,5.0,1.0,2. Get surgery for cleft lips at 3 to 6 months...,cleft lips,PLAUSIBLE,5.0
3,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear,NEUTRAL,IMPLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,3.0,1.5,5.0,5.0,1.0,2. Get surgery for your baby at 3 to 6 months ...,your baby,PLAUSIBLE,5.0
4,0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear,NEUTRAL,IMPLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,3.0,1.5,5.0,5.0,1.0,2. Get surgery for the ear at 3 to 6 months old.,the ear,IMPLAUSIBLE,1.0


In [ ]:
df_single = pd.read_csv(project_path+'Datasets/Clarification_Merge_Single_Filler_Test_1.csv')

In [ ]:
sent_len = []
for i in range(len(df_1['Id'])):
  sent = str(df_1['Article title'][i])+' '+ str(df_1['Section header'][i]) +' '+str(df_1['Previous context'][i])+' '+str(df_1['Sentence'][i])+' '+str(df_1['Follow-up context'][i])
  
  sent_len.append(len(sent.split()))

print(max(sent_len))

184


In [ ]:
#from transformers import BertTokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer.special_tokens_map
'''
{'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'pad_token': '[PAD]',
 'sep_token': '[SEP]',
 'unk_token': '[UNK]'}
'''

In [ ]:

merge_text_cols = ['Article title', 'Section header',
       'Previous context', 'Filled_Sentence', 'Follow-up context']

#text_special_tokens = ['', '[SEP]','[SEP]','[SEP]','[SEP]']
text_special_tokens = ['', '','','','']


In [ ]:
class_names = ["IMPLAUSIBLE", "NEUTRAL", "PLAUSIBLE"]

LABEL_DICT = {}
LABEL_DICT[class_names[0]] = 0
LABEL_DICT[class_names[1]] = 1
LABEL_DICT[class_names[2]] = 2

In [7]:
import re
pattern_1 = "(...)"
pattern_2 = "..."
def preprocess_text(text):
  text = re.sub(r"\*","",text)
  text = re.sub(r"\(\.\.\.\)","",text)
  text = re.sub(r"\.\.\.","",text)
  text = re.sub(r"______"," [BLANK] ",text)
  text = re.sub(r"\s{2,}", " ", text)

  return text

In [ ]:
df_merge={}
for c in df_single.columns:
  df_merge[c] = []

df_merge['Text']=[]
df_merge['Text_with_Filler_Token']=[]
df_merge['LabelNum'] = []

for i in range(5):
  df_merge['scoreRank_'+str(i+1)] = []

df_merge['ScoreRank'] = []

for i in range(len(df_single)):
  for c in df_single.columns:
    df_merge[c].append(df_single[c][i])

  text = ''
  filler = df_single['Filler'][i]
  for j in range(len(merge_text_cols)):
    text+= text_special_tokens[j]+' '+str(df_single[merge_text_cols[j]][i])+' '
    text = preprocess_text(text)
  df_merge['Text'].append(text)
  text_with_Sp = text.replace(str(filler)," "+str(filler)+" ")
  text_with_Sp = preprocess_text(text_with_Sp)
  df_merge['Text_with_Filler_Token'].append(text_with_Sp)
  df_merge['LabelNum'].append(LABEL_DICT[str(df_single['Label'][i])])

  temp_score = []
  for k in range(1,6):
    temp_score.append(float(df_single['score_'+str(k)][i]))
  temp_score_index = [sorted(temp_score).index(x) for x in temp_score]

  for k in range(1,6):
    df_merge['scoreRank_'+str(k)].append(temp_score_index[k-1])
  df_merge['ScoreRank'].append(temp_score_index[i%5])



df_merge = pd.DataFrame(df_merge)
df_merge.to_csv(project_path+'Datasets/Clarification_Merge_Single_Filler_Text_Test_1_1.csv',index=False)

print(len(df_merge),len(df_single))
df_merge.tail()

2500 2500


,Id,Resolved pattern,Article title,Section header,Previous context,Sentence,Follow-up context,Filler1,Filler2,Filler3,Filler4,Filler5,label_1,label_2,label_3,label_4,label_5,score_1,score_2,score_3,score_4,score_5,Filled_Sentence,Filler,Label,Score,Text,Text_with_Filler_Token,LabelNum,scoreRank_1,scoreRank_2,scoreRank_3,scoreRank_4,scoreRank_5,ScoreRank
2495,499,FUSED HEAD,How to Grow Houseplants,Learning About Future Care,1. Learn how good culture prevents plant stre...,A plant will likely never have any serious ___...,Sometimes bad things can happen.,troubles,parasites,illness,discomfort,difficulties,PLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,PLAUSIBLE,5.0,5.0,5.0,2.5,5.0,A plant will likely never have any serious tro...,troubles,PLAUSIBLE,5.0,How to Grow Houseplants Learning About Future...,How to Grow Houseplants Learning About Future...,2,1,1,1,0,1,1
2496,499,FUSED HEAD,How to Grow Houseplants,Learning About Future Care,1. Learn how good culture prevents plant stre...,A plant will likely never have any serious ___...,Sometimes bad things can happen.,troubles,parasites,illness,discomfort,difficulties,PLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,PLAUSIBLE,5.0,5.0,5.0,2.5,5.0,A plant will likely never have any serious par...,parasites,PLAUSIBLE,5.0,How to Grow Houseplants Learning About Future...,How to Grow Houseplants Learning About Future...,2,1,1,1,0,1,1
2497,499,FUSED HEAD,How to Grow Houseplants,Learning About Future Care,1. Learn how good culture prevents plant stre...,A plant will likely never have any serious ___...,Sometimes bad things can happen.,troubles,parasites,illness,discomfort,difficulties,PLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,PLAUSIBLE,5.0,5.0,5.0,2.5,5.0,A plant will likely never have any serious ill...,illness,PLAUSIBLE,5.0,How to Grow Houseplants Learning About Future...,How to Grow Houseplants Learning About Future...,2,1,1,1,0,1,1
2498,499,FUSED HEAD,How to Grow Houseplants,Learning About Future Care,1. Learn how good culture prevents plant stre...,A plant will likely never have any serious ___...,Sometimes bad things can happen.,troubles,parasites,illness,discomfort,difficulties,PLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,PLAUSIBLE,5.0,5.0,5.0,2.5,5.0,A plant will likely never have any serious dis...,discomfort,IMPLAUSIBLE,2.5,How to Grow Houseplants Learning About Future...,How to Grow Houseplants Learning About Future...,0,1,1,1,0,1,0
2499,499,FUSED HEAD,How to Grow Houseplants,Learning About Future Care,1. Learn how good culture prevents plant stre...,A plant will likely never have any serious ___...,Sometimes bad things can happen.,troubles,parasites,illness,discomfort,difficulties,PLAUSIBLE,PLAUSIBLE,PLAUSIBLE,IMPLAUSIBLE,PLAUSIBLE,5.0,5.0,5.0,2.5,5.0,A plant will likely never have any serious dif...,difficulties,PLAUSIBLE,5.0,How to Grow Houseplants Learning About Future...,How to Grow Houseplants Learning About Future...,2,1,1,1,0,1,1


In [8]:
df_merge = pd.read_csv(project_path+'Datasets/Clarification_Merge_Single_Filler_Text_Test_1_1.csv')

In [9]:
print(df_merge['Text_with_Filler_Token'][0])

 How to Detect Cleft Lip Early in Pregnancy Treating Cleft Lip 1. Care for a newborn with a cleft lip. You'll also get help with feeding the baby since they may have trouble sucking or swallowing. 2. Get surgery for the split at 3 to 6 months old. A pediatric surgeon will move the skin and muscles around your baby's lips to realign them. 


In [10]:
df_all_filler = {}

df_all_filler['Text_with_options'] = []
df_all_filler['ScoreRankList']=[]

for i in range(0,len(df_merge['Filled_Sentence']),5):
  ss = str(df_merge['Article title'][i]) +' ' + str(df_merge['Previous context'][i])
  scoreRankList = []
  for fillerNum in range(0,5):
    ss += ' [OPTION_'+str(fillerNum+1)+'] '+str(df_merge['Filled_Sentence'][i+fillerNum])
    scoreRankList.append(df_merge['ScoreRank'][i+fillerNum])

  ss += ' [FOLLOW] '+str(df_merge['Follow-up context'][i])
  ss = preprocess_text(ss)

  df_all_filler['Text_with_options'].append(ss)
  df_all_filler['ScoreRankList'].append(scoreRankList)

df_all_filler = pd.DataFrame(df_all_filler)
df_all_filler.to_csv(project_path+'Datasets/Clarification_Merge_All_Filler_Text_Test_1.csv',index=False)
print(len(df_all_filler['Text_with_options']))


500


In [ ]:
sent_len = []
for i in range(len(df_merge['Id'])):
  sent_len.append(len(df_merge['Text_with_Filler_Token'][i].split()))

print(max(sent_len))

204


In [ ]:
len(df_1)#['score_5'][15]

500

17959

In [ ]:
df_train['ScoreRank']

In [ ]:
print(df_scores.tail())

           Id  Scores
19970  3995_1     1.0
19971  3995_2     5.0
19972  3995_3     1.0
19973  3995_4     3.0
19974  3995_5     2.0


In [ ]:
print(df_test['scoreRank_5'][399])

2


In [ ]:
df_data.tail()

In [ ]:
'''
df_train = {}
df_val = {}
df_test = {}

for c in df_1.columns:
  df_train[c] = []
  df_val[c] = []
  df_test[c] = []

for i_train in range(TRAIN_LENGTH):
  for c in df_1.columns:
    df_train[c].append(df_1[c][i_train])

for i_val in range(TRAIN_LENGTH, TRAIN_LENGTH+VAL_LENGTH):
  for c in df_1.columns:
    df_val[c].append(df_1[c][i_val])

for i_test in range(TRAIN_LENGTH+VAL_LENGTH, TRAIN_LENGTH+VAL_LENGTH+TEST_LENGTH):
  for c in df_1.columns:
    df_test[c].append(df_1[c][i_test])

df_train = pd.DataFrame(df_train)
df_train.to_csv(project_path+'Datasets/Clarification_INIT_Train_Merge_1.csv',index=False)

df_val = pd.DataFrame(df_val)
df_val.to_csv(project_path+'Datasets/Clarification_INIT_Valid_Merge_1.csv',index=False)

df_test = pd.DataFrame(df_test)
df_test.to_csv(project_path+'Datasets/Clarification_INIT_Test_Merge_1.csv',index=False)

print(TRAIN_LENGTH, VAL_LENGTH, TEST_LENGTH, (TRAIN_LENGTH+VAL_LENGTH+TEST_LENGTH), len(df_1))
print(len(df_train), len(df_val), len(df_test), (len(df_train)+len(df_val)+len(df_test)), len(df_1))
'''

3192 399 400 3991 3991
3192 399 400 3991 3991


In [ ]:
print(df_merge_2.columns)

Index(['Id', 'Resolved pattern', 'Article title', 'Section header',
       'Previous context', 'Sentence', 'Follow-up context', 'Filler1',
       'Filler2', 'Filler3', 'Filler4', 'Filler5', 'label_1', 'label_2',
       'label_3', 'label_4', 'label_5', 'score_1', 'score_2', 'score_3',
       'score_4', 'score_5', 'Text', 'Text_With_Filler', 'Text_Score',
       'Text_Rearr', 'Text_With_Filler_Rearr', 'Clean_Text',
       'Clean_Text_With_Filler', 'Clean_Text_Rearr',
       'Clean_Text_With_Filler_Rearr', 'labelNum_1', 'labelNum_2',
       'labelNum_3', 'labelNum_4', 'labelNum_5', 'scoreRank_1', 'scoreRank_2',
       'scoreRank_3', 'scoreRank_4', 'scoreRank_5'],
      dtype='object')


In [ ]:
print(df_test['Text_Score'][399])

[SCORE1] 1.0 [SCORE2] 5.0 [SCORE3] 1.0 [SCORE4] 3.0 [SCORE5] 2.0 


In [ ]:
print(np.max([len(tt) for tt in df_all['Sentence']]))

1976


In [ ]:
#!pip install spicy
import spicy
from scipy import stats
a = [4,4,2,3.5,4.5,4.5]
b = [2, 2, 0, 1, 5, 5]
print(stats.spearmanr(a,b)[0])
#print(float('6.656678897778'))

0.9999999999999999


In [ ]:
a = [4,4,2,3.5,4.5,4.5]
b = a.copy()
print([sorted(a).index(x) for x in a])


[2, 2, 0, 1, 4, 4]


In [ ]:
import re

def remove_snake_case(data):
  tokenized = data.split('_')
  return ' '.join(tokenized)

def remove_camel_case(data):
  tokenized = data.split()
  new_tokens =[]

  if len(tokenized) <= 2:
      return ' '.join(tokenized)

  for t in tokenized:
    match = re.match('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', t)
    if match.group(0) is not t:
      new_tokens.append(match.group(0))
      temp = t.replace(match.group(0),'')
      new_token.append(temp)
    else:
      new_tokens.append(t)
  
  return ' '.join(new_tokens)


def clean_data(data):
  tokens = remove_snake_case(data)
  tokens = remove_camel_case(tokens)

  return tokens


In [ ]:
class Example(object):
    """A single training/test example."""
    def __init__(self,
                 idx,
                 source,
                 target,
                 ):
        self.idx = idx
        self.source = source
        self.target = target



def read_examples(df,source_col='news',target_col='kb'):
    """Read examples from filename."""
    examples=[]
    src_list = df[source_col].tolist()
    tgt_list = df[target_col].tolist()
    idx = 0

    assert len(src_list)==len(tgt_list)

    for i in range(len(src_list)):
        #line1 = clean_data(line1)
        #line2 = clean_data(line2)
        examples.append(
        Example(
                idx = idx,
                source=src_list[i].strip(),
                target=tgt_list[i].strip(),
                ) 
        )
        idx+=1
    return examples

In [ ]:
df_train=pd.read_csv(project_path+'Datasets/Dataset_Gun_KB_train_1.csv')

examples = read_examples(df_train,source_col='news',target_col='kb')

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_train_gun_kb_1.pickle', 'wb') as f:
  pickle.dump(examples,f)
  #pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle
with open(project_path+'Pickles/pickle_train_gun_kb_1.pickle', 'rb') as f:
  train_examples = pickle.load(f)

In [ ]:
print(train_examples[4000].source)
print(train_examples[4000].target)

Title: Deputies: Man planned to kill 2 others, himself
Detail: ST. JOHNS COUNTY, Fla. – A 24-year-old Palm Coast man was arrested Sunday on felony charges of fleeing and attempting to elude a law enforcement officer and aggravated stalking, with additional charges pending further investigation.

Deputies in Flagler County said they learned Sunday afternoon that Andrew Dandurand had armed himself with a possible stolen gun and was planning to kill his ex-girlfriend, her current boyfriend and himself.

Deputies responded to Dandurand's ex-girlfriend's home in St. Johns County, where she was found unharmed and was aware of the threats made against her.

Investigators said they also learned that Dandurand possibly had an assault rifle. Deputies found him about a mile from his ex-girlfriend's home, where he tried to flee from a traffic stop.

Dandurand drove recklessly over an embankment, ran a red light and drove southbound in the northbound lane on U.S. 1, deputies said.

Deputies perform

In [ ]:
df_val=pd.read_csv(project_path+'Datasets/Dataset_Gun_KB_valid_1.csv')

examples = read_examples(df_val,source_col='news',target_col='kb')

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_val_gun_kb_1.pickle', 'wb') as f:
  pickle.dump(examples,f)
  #pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle
with open(project_path+'Pickles/pickle_val_gun_kb_1.pickle', 'rb') as f:
  val_examples = pickle.load(f)

In [ ]:
print(val_examples[4000].source)
print(val_examples[4000].target)

Title: Arrest made in theft of firearm during Oroville evacuations
Detail: Oroville >> An Oroville man on parole was arrested Monday after officers found he was allegedly in possession of a firearm that had been stolen from a home during evacuations that had been ordered below Oroville Dam, police said.

The man, Kayton Clark, 40, was allegedly found to be in possession of a .30-30 rifle and a crossbow when officers searched his home about 10 a.m. Monday in the 2200 block of Perkins Avenue in Oroville, according to a press release.

Oroville police and agents for the California Department of Corrections and Rehabilitation searched Clark’s home as part of a parole compliance check for Clark, according to the release.

The authorities learned that the rifle had been stolen from a nearby residence, according to the release.

Officers spoke with the victim of the firearm theft and learned the rifle was taken during a home burglary that happened during the time residents had been ordered to

In [ ]:
df_test=pd.read_csv(project_path+'Datasets/Dataset_Gun_KB_test_1.csv')

examples = read_examples(df_test,source_col='news',target_col='kb')

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_test_gun_kb_1.pickle', 'wb') as f:
  pickle.dump(examples,f)
  #pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle
with open(project_path+'Pickles/pickle_test_gun_kb_1.pickle', 'rb') as f:
  test_examples = pickle.load(f)

In [ ]:
print(test_examples[4000].source)
print(test_examples[4000].target)

Title: Man accused of shooting at officers in Green Twp. appears in court
Detail: Man accused of shooting at officers in Green Twp. appears in court

A man accused of shooting a gun at officers and his neighbor appeared in court Monday. Brendon MacDonald, 51, is charged with two counts of attempted murder and five counts of felonious assault after police said he shot at them from his Green Township home Sunday morning. A Jessup Road resident called police at about 7 a.m. to report that his neighbor had shot at him. The neighbor said MacDonald was in the street talking about "killing demons" before he fired at him. “There’s a guy out in the, street, talking about killing demons. I went down and asked him if he was alright, and a neighbor across the street. He pulled a gun out," the caller said. When officers arrived, police said that MacDonald was sitting in a lawn chair on his front porch. MacDonald stood up and fired at the officers, they said. Two officers returned fire but did not s

In [ ]:
import logging

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask       
        


def convert_examples_to_features(examples, source_tokenizer, target_tokenizer, max_source_length = 512, max_target_length=512, padding='max_length',truncation=True,prefix=None,target_prefix = None, stage=None):
    features = []
    if(prefix is None):
      prefix=''
    
    if(target_prefix is None):
      target_prefix=''

    for example_index, example in enumerate(examples):
        #source
        source_tokenized = source_tokenizer(prefix+example.source, max_length=max_source_length,padding=padding,truncation=truncation)
        source_ids =  source_tokenized['input_ids']
        source_mask = source_tokenized['attention_mask']
      

        with target_tokenizer.as_target_tokenizer():
          target_tokenized = target_tokenizer(target_prefix+example.target, max_length=max_target_length,padding=padding,truncation=truncation)          
          target_ids = target_tokenized['input_ids']
          target_mask = target_tokenized['attention_mask']
 
   
        if example_index < 3:
            #if stage=='train':
            logging.info("*** Example ***")
            logging.info("idx: {}".format(example.idx))

            logging.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokenizer.convert_ids_to_tokens(source_ids)]))
            logging.info("source_ids: {}".format(' '.join(map(str, source_ids))))
            logging.info("source_mask: {}".format(' '.join(map(str, source_mask))))
            
            logging.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokenizer.convert_ids_to_tokens(target_ids)]))
            logging.info("target_ids: {}".format(' '.join(map(str, target_ids))))
            logging.info("target_mask: {}".format(' '.join(map(str, target_mask))))
       
        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features

In [ ]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 35.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.2 MB/s 
     |████████████████████████████████| 636 kB 44.4 MB/s 
     |████████████████████████████████| 1.1 MB 40.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")

special_tokens_dict = {'additional_special_tokens': ['[START]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#model.resize_token_embeddings(len(tokenizer))

In [ ]:
'''
decoder_tokenizer.cls_token = decoder_tokenizer.bos_token
decoder_tokenizer.sep_token = decoder_tokenizer.eos_token
decoder_tokenizer.pad_token = decoder_tokenizer.eos_token
encoder_tokenizer.pad_token = decoder_tokenizer.eos_token
'''

print(tokenizer.cls_token,tokenizer.sep_token,tokenizer.bos_token,tokenizer.eos_token,tokenizer.pad_token,tokenizer.sep_token_id,tokenizer.pad_token_id)
print(tokenizer.special_tokens_map)
#print(decoder_tokenizer.cls_token,decoder_tokenizer.sep_token,decoder_tokenizer.bos_token,decoder_tokenizer.eos_token,decoder_tokenizer.pad_token,decoder_tokenizer.sep_token_id,decoder_tokenizer.pad_token_id)

Using cls_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.


None None None </s> <pad> None 0
{'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['[START]']}


In [ ]:
train_features = convert_examples_to_features(train_examples,tokenizer,tokenizer,max_source_length=512,max_target_length=512,padding='max_length',truncation=True,prefix='crime: ',target_prefix='[START]')

In [ ]:
print(len(train_features[2000].source_ids))

512


In [ ]:
print(tokenizer.decode(train_features[2000].source_ids))

crime: Title: Suspect jumps in lake after firing off two shots at Civic Center Detail: Joshua Wright. (Source: Calcasieu Parish Sheriff's Office) LAKE CHARLES, LA (KPLC) - A Lake Charles man was arrested late Saturday night after firing a gun outside the Lake Charles Civic Center, authorities said. Two Calcasieu Parish Sheriff's Office deputies were driving near the Civic Center around 11 p.m. when they heard gun shots. According to deputies, there was a disturbance in the parking lot, which led 23-year-old Joshua Wright to fire two shots into the air. However, witnesses claim only one shot was fired. Commander James McGee said Wright threw the handgun in the lake and jumped in the water. McGee said Wright returned a short time later and was arrested. No one was injured in the incident. Wright has been charged with two counts of aggravated assault, obstruction of justice and resisting an officer. Lake Charles Police also assisted in the call. Copyright 2014 KPLC. All rights reserved.</

In [ ]:
print(train_features[0].target_ids)
print(train_features[300].target_ids)
print(tokenizer.decode(train_features[0].target_ids))

[32100, 1923, 5628, 7, 10, 3, 13114, 12060, 2998, 7, 10, 41, 279, 49, 29, 986, 18584, 159, 6, 16047, 12672, 1361, 3670, 3, 13114, 41, 14318, 99, 127, 29, 23, 9, 6, 1626, 210, 17, 6293, 15, 6, 1179, 2560, 2463, 13, 1064, 7854, 6916, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_train_gun_kb_features_1.pickle', 'wb') as f:
  pickle.dump(train_features,f)
  #pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle

with open(project_path+'Pickles/pickle_train_gun_kb_features_1.pickle', 'rb') as f:
  train_features = pickle.load(f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
val_features = convert_examples_to_features(val_examples,tokenizer,tokenizer,max_source_length=512,max_target_length=512,padding='max_length',truncation=True,prefix='crime: ')

In [ ]:
print(len(val_features[0].source_ids))

512


In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_gun_kb_features_1.pickle', 'wb') as f:
  pickle.dump(val_features,f)
  #pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_gun_kb_features_1.pickle', 'rb') as f:
  val_features = pickle.load(f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
test_features = convert_examples_to_features(test_examples,tokenizer,tokenizer,max_source_length=512,max_target_length=512,padding='max_length',truncation=True,prefix='crime: ')

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_test_gun_kb_features_1.pickle', 'wb') as f:
  pickle.dump(test_features,f)
  #pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle

with open(project_path+'Pickles/pickle_test_gun_kb_features_1.pickle', 'rb') as f:
  test_features = pickle.load(f) 
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels =[[-100 if token == tokenizer.pad_token_id else token for token in ll] for ll in labels]
        self.temp = None

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
print(train_features[0].target_ids)
print(train_features[0].target_ids[:-1])
print(train_features[0].target_ids[1:5]+[tokenizer.pad_token_id])

[32100, 1923, 5628, 7, 10, 3, 13114, 12060, 2998, 7, 10, 41, 279, 49, 29, 986, 18584, 159, 6, 16047, 12672, 1361, 3670, 3, 13114, 41, 14318, 99, 127, 29, 23, 9, 6, 1626, 210, 17, 6293, 15, 6, 1179, 2560, 2463, 13, 1064, 7854, 6916, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
train_dict = {}
train_dict["input_ids"] = []
train_dict["attention_mask"] = []
train_dict["decoder_input_ids"] = []
train_dict["decoder_attention_mask"] = []
train_labels = []

for e in train_features:
  train_dict["input_ids"].append(e.source_ids)
  train_dict["attention_mask"].append(e.source_mask)
  train_dict["decoder_input_ids"].append(e.target_ids)
  train_dict["decoder_attention_mask"].append(e.target_mask)
  train_labels.append(e.target_ids[1:]+[tokenizer.pad_token_id])  #####################


train_dataset = CustomDataset(train_dict,train_labels)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_train_gun_kb_dataset_1.pickle', 'wb') as f:
  pickle.dump(train_dataset,f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_train_gun_kb_dataset_1.pickle', 'rb') as f:
  train_dataset = pickle.load(f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
print(train_dataset.labels[0])

[1923, 5628, 7, 10, 3, 13114, 12060, 2998, 7, 10, 41, 279, 49, 29, 986, 18584, 159, 6, 16047, 12672, 1361, 3670, 3, 13114, 41, 14318, 99, 127, 29, 23, 9, 6, 1626, 210, 17, 6293, 15, 6, 1179, 2560, 2463, 13, 1064, 7854, 6916, 61, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [ ]:
val_dict = {}
val_dict["input_ids"] = []
val_dict["attention_mask"] = []
#val_dict["decoder_input_ids"] = []
#val_dict["decoder_attention_mask"] = []

val_labels = []

for e in val_features:
  val_dict["input_ids"].append(e.source_ids)
  val_dict["attention_mask"].append(e.source_mask)
  #val_dict["decoder_input_ids"].append(e.target_ids)
  #val_dict["decoder_attention_mask"].append(e.target_mask)

  val_labels.append(e.target_ids)


val_dataset = CustomDataset(val_dict,val_labels)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_gun_kb_dataset_1.pickle', 'wb') as f:
  pickle.dump(val_dataset,f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_gun_kb_dataset_1.pickle', 'rb') as f:
  val_dataset = pickle.load(f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
print(val_dataset[5])

In [ ]:
test_dict = {}
test_dict["input_ids"] = []
test_dict["attention_mask"] = []

test_labels = []

for e in test_features:
  test_dict["input_ids"].append(e.source_ids)
  test_dict["attention_mask"].append(e.source_mask)

  test_labels.append(e.target_ids)


test_dataset = CustomDataset(test_dict,test_labels)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_test_gun_kb_dataset_1.pickle', 'wb') as f:
  pickle.dump(test_dataset,f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_test_gun_kb_dataset_1.pickle', 'rb') as f:
  test_dataset = pickle.load(f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
val_dict = {}
val_dict["input_ids"] = []
val_dict["attention_mask"] = []
#val_dict["decoder_input_ids"] = []
#val_dict["decoder_attention_mask"] = []

val_labels = []

for e in val_features[:5]:
  val_dict["input_ids"].append(e.source_ids)
  val_dict["attention_mask"].append(e.source_mask)
  #val_dict["decoder_input_ids"].append(e.target_ids)
  #val_dict["decoder_attention_mask"].append(e.target_mask)

  val_labels.append(e.target_ids)


val_small_dataset = CustomDataset(val_dict,val_labels)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_small_gun_kb_dataset_1.pickle', 'wb') as f:
  pickle.dump(val_small_dataset,f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
import random
random.seed(42)

In [ ]:
print(val_features[0].example_id)
shuffled_val_features = random.sample(val_features,1000) ################################
print(shuffled_val_features[0].example_id)

0
5238


In [ ]:
shuffled_val_dict = {}
shuffled_val_dict["input_ids"] = []
shuffled_val_dict["attention_mask"] = []
#shuffled_val_dict["decoder_input_ids"] = []
#shuffled_val_dict["decoder_attention_mask"] = []

shuffled_val_labels = []

for e in shuffled_val_features:
  shuffled_val_dict["input_ids"].append(e.source_ids)
  shuffled_val_dict["attention_mask"].append(e.source_mask)
  #shuffled_val_dict["decoder_input_ids"].append(e.target_ids)
  #shuffled_val_dict["decoder_attention_mask"].append(e.target_mask)

  shuffled_val_labels.append(e.target_ids)


shuffled_val_dataset = CustomDataset(shuffled_val_dict,shuffled_val_labels)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_gun_kb_shuffled_1000_dataset_1.pickle', 'wb') as f:
  pickle.dump(shuffled_val_dataset,f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_valid_gun_kb_shuffled_1000_dataset_1.pickle', 'rb') as f:
  shuffled_val_dataset = pickle.load(f)
  #pickle.dump((test_df['labels'],predictions),f)

In [ ]:
print(tokenizer.decode(shuffled_val_dataset.encodings['input_ids'][0]))
#print(shuffled_val_dataset.labels[0])

crime: Title: Woman accused of pointing gun during road-rage incident Detail: ORLANDO, Fla. – A woman who had her infant son with her is accused of pointing a gun at another driver during a road-rage incident Sunday in Orlando, according to the Orlando Police Department. The victim said he pulled his red Dodge onto Conroy Road and was nearly hit by a silver Jeep that was in his blind spot. The driver of the Jeep sped up then braked to avoid hitting the Dodge and both vehicles continued down the road, authorities said. The victim and his passenger had lunch at a nearby restaurant, and when they tried to leave, the family from the silver Jeep stood in front of the Dodge to prevent it from leaving, police said. The man said, "You about hit my car" and Kayla Nogues, 24, passed her infant son to her mother and started to yell at the man, according to the arrest affidavit. Nogues grabbed a small black handgun from her backpack, pointing it directly at the victims and telling them she would "

In [ ]:
from transformers import EncoderDecoderModel

ed = EncoderDecoderModel.from_encoder_decoder_pretrained("microsoft/graphcodebert-base","microsoft/CodeGPT-small-java-adaptedGPT2")

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at microsoft/CodeGPT-small-java-adaptedGPT2 and are newly initialized: ['transformer.h.1.crossattention.c_proj.bias', 'transformer.h.0.crossattention.bias', 'transformer.h.11.crossattention.c_proj.weight', 'transformer.h.11.ln_cross_attn.weight', 'transformer.h.2.crossattention.c_proj.bias', 'transformer.h.5.ln_cross_attn.weight', 'transformer.h.7.crossattention.masked_bias', 'transformer.h.4.ln_cross_attn.weight', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.4.crossattention.bias', 'transformer.h.0.crossattention.masked_bias', 'transformer.h.9.crossattention.q_attn.weight', 'transformer.h.6.crossattention.q_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.9.crossattention.bias', 'transformer.h.6.crossattention.c_attn.weight', 'transformer.h.11.crossattention.c_proj.bias', 'transformer.h.1.crossattention.bias', 'transformer.h.5.crossattention.masked_bias', 'transform

# **TESTING.....**

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

batch_size = 2

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=100,
    eval_steps=400,
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=ed,
    #tokenizer=tokenizer,
    args=training_args,
    #compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:

ed.config.decoder_start_token_id = decoder_tokenizer.cls_token_id
ed.config.eos_token_id = decoder_tokenizer.sep_token_id
ed.config.pad_token_id = decoder_tokenizer.pad_token_id
ed.config.vocab_size = ed.config.encoder.vocab_size


op = ed.generate(input_ids=train_dataset[:2]['input_ids'].to("cuda"),attention_mask=train_dataset[:2]['attention_mask'].to("cuda"))
#print(train_dataset[0]['input_ids'])

output_str = decoder_tokenizer.batch_decode(op,skip_special_tokens=False)
print(output_str)


['<s>public static void METHOD_1 ( java.lang.String VAR_1 ) {', '<s>. METHOD_1 ( ) ) ) { if (! ( VAR_1.']


In [ ]:
ed.save_pretrained(project_path+'Saved_models/ED_1')

In [ ]:
from transformers import EncoderDecoderModel
ed = EncoderDecoderModel.from_pretrained(project_path+'Saved_models/ED_1')

In [ ]:
print(decoder_tokenizer.decode(ed.config.pad_token_id))

</s>


In [ ]:
ss = decoder_tokenizer.batch_decode(test_features[:1].target_ids,skip_special_tokens=True)
print(ss)
for s in ss:
  print(s)

AttributeError: ignored

In [ ]:
tt = AutoTokenizer.from_pretrained('gpt2')
print(tt.special_tokens_map)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}


In [ ]:
print(train_dataset[:1]['input_ids'].size())

torch.Size([1, 256])


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

special_tokens_dict = {'additional_special_tokens': ['[prob_statement]','[input_statement]','[output_statement]','[sample_example]','[problem_note]','[prog_language]','[user_rating]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#model.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
#tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
#tokenizer = AutoTokenizer.from_pretrained("microsoft/CodeGPT-small-java")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("microsoft/graphcodebert-base")



Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
code_tokenized=tokenizer([tokenizer.bos_token+"def FUNC_2(a,b): if a> TYPE_3: return VAR_1 else return camelCase"+tokenizer.eos_token,tokenizer.bos_token+"def FUNC_2(a,b): if a> TYPE_3: return sdfdsf ertert trytry tyutyu VAR_1 else return camelCase"+tokenizer.eos_token],max_length=512,padding=True)

print(code_tokenized['attention_mask'][1])
print(tokenizer.decode(code_tokenized['input_ids'][1],skip_special_tokens=False))
#print(tokenizer.convert_ids_to_tokens(code_tokenized['input_ids']))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|endoftext|>def FUNC_2(a,b): if a> TYPE_3: return sdfdsf ertert trytry tyutyu VAR_1 else return camelCase<|endoftext|>


In [ ]:
print(tokenizer.eos_token)

</s>


In [ ]:
'''
import pickle

with open(project_path+'Pickles/pickle_clean_sentences_1.pickle', 'wb') as f:
  pickle.dump(clean_sentences,f)
  #pickle.dump((test_df['labels'],predictions),f)
'''

In [ ]:
import pickle
with open(project_path+'Pickles/pickle_clean_sentences_1.pickle', 'rb') as f:
  sentences = pickle.load(f)

In [ ]:
print(len(sentences))

36565


In [ ]:
import re

comment_remover = '(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)'

#temp = 'char *note = "You can make a one-line comment with //";'

clean_code_data = []

for i in range(len(code_data[:19000])):
  #if(i<20000):
    #continue
  #if(i%1000==0):
    #pass
  #print(str(i)+'/'+str(len(code_data)))
  temp = code_data[i]
  #print(temp)
  temp = re.sub(comment_remover, '', temp)

  clean_code_data.append(temp)

print(clean_code_data[0])
print(len(clean_code_data))

In [ ]:
import re

comment_remover = '(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)'

#temp = 'char *note = "You can make a one-line comment with //";'

clean_code_data2 = []

for i in range(19000,len(code_data)):
  #if(i<20000):
    #continue
  if(i%1000==0):
    print(str(i)+'/'+str(len(code_data)))
  temp = code_data[i]
  #print(temp)
  try:
    temp = re.sub(comment_remover, '', temp)
  except:
    print('struck')

  clean_code_data2.append(temp)

print(clean_code_data2[0])
print(len(clean_code_data2))

In [ ]:
clean_code_data_all = []

for i in range(len(clean_code_data)):
  clean_code_data_all.append(clean_code_data[i])

for i in range(len(clean_code_data2)):
  clean_code_data_all.append(clean_code_data2[i])

print(len(clean_code_data_all))

NameError: ignored

In [ ]:
'''
import pickle

with open(project_path+'Pickles/pickle_clean_codes_1.pickle', 'wb') as f:
  pickle.dump(clean_code_data_all,f)
  #pickle.dump((test_df['labels'],predictions),f)
'''

In [ ]:
import pickle
with open(project_path+'Pickles/pickle_clean_codes_1.pickle', 'rb') as f:
  code_data_all = pickle.load(f)

In [ ]:
print(len(code_data_all))
print(code_data_all[-3])

In [ ]:
from transformers import LongformerModel, LongformerTokenizer

encoder_tokenizer = LongformerTokenizer.from_pretrained("microsoft/codebert-base")

special_tokens_dict = {'additional_special_tokens': ['[prob_statement]','[input_statement]','[output_statement]','[sample_example]','[problem_note]','[prog_language]','[user_rating]']}
num_added_toks = encoder_tokenizer.add_special_tokens(special_tokens_dict)


encoder_model = LongformerModel.from_pretrained("microsoft/codebert-base")
encoder_model.resize_token_embeddings(len(encoder_tokenizer))

encoder_model.save_pretrained(project_path+'Saved_Models/Longformer_Encoder_Init')


You are using a model of type roberta to instantiate a model of type longformer. This is not supported for all configurations of models and can yield errors.


Some weights of LongformerModel were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['encoder.layer.5.attention.self.key_global.weight', 'encoder.layer.4.attention.self.query_global.bias', 'encoder.layer.2.attention.self.value_global.weight', 'encoder.layer.5.attention.self.value_global.weight', 'encoder.layer.8.attention.self.value_global.bias', 'encoder.layer.1.attention.self.key_global.weight', 'encoder.layer.3.attention.self.key_global.bias', 'encoder.layer.2.attention.self.key_global.bias', 'encoder.layer.4.attention.self.value_global.bias', 'encoder.layer.8.attention.self.query_global.weight', 'encoder.layer.2.attention.self.query_global.bias', 'encoder.layer.3.attention.self.key_global.weight', 'encoder.layer.3.attention.self.value_global.weight', 'encoder.layer.2.attention.self.value_global.bias', 'encoder.layer.7.attention.self.query_global.bias', 'encoder.layer.1.attention.self.value_global.weight', 'encoder.layer.8.attention.s

In [ ]:
from transformers import LongformerforSeq2, LongformerTokenizer

decoder_tokenizer = LongformerTokenizer.from_pretrained("microsoft/codebert-base")

special_tokens_dict = {'additional_special_tokens': ['[problem_tags]','[problem_difficulty]','[req_time]','[req_memory]','[code_text]']}
num_added_toks = encoder_tokenizer.add_special_tokens(special_tokens_dict)


decoder_model = LongformerLMHeadModel.from_pretrained("microsoft/codebert-base")
decoder_model.resize_token_embeddings(len(decoder_tokenizer))

decoder_model.save_pretrained(project_path+'Saved_Models/LongformerLMHead_Decoder_Init')


In [ ]:
from transformers import AutoModelForSeq2SeqLM

led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384")

In [ ]:
def tensorify(lst):
    """
    List must be nested list of tensors (with no varying lengths within a dimension).
    Nested list of nested lengths [D1, D2, ... DN] -> tensor([D1, D2, ..., DN)

    :return: nested list D
    """
    # base case, if the current list is not nested anymore, make it into tensor
    if type(lst[0]) != list:
        if type(lst) == torch.Tensor:
            return lst
        elif type(lst[0]) == torch.Tensor:
            return torch.stack(lst, dim=0)
        else:  # if the elements of lst are floats or something like that
            return torch.tensor(lst)
    current_dimension_i = len(lst)
    for d_i in range(current_dimension_i):
        tensor = tensorify(lst[d_i])
        lst[d_i] = tensor
    # end of loop lst[d_i] = tensor([D_i, ... D_0])
    tensor_lst = torch.stack(lst, dim=0)
    return tensor_lst

In [ ]:
from transformers import LongformerModel, LongformerTokenizer, AutoTokenizer

led_tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

special_tokens_dict = {'additional_special_tokens': ['[prob_statement]','[input_statement]','[output_statement]','[sample_example]','[problem_note]','[prog_language]','[user_rating]','[problem_tags]','[problem_difficulty]','[req_time]','[req_memory]','[code_text]']}
num_added_toks = led_tokenizer.add_special_tokens(special_tokens_dict)



In [ ]:
'''
from transformers import LongformerModel, LongformerTokenizer, AutoModelForSeq2SeqLM

led_model = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)
led_model.resize_token_embeddings(len(led_tokenizer))

led_model.save_pretrained(project_path+'Saved_Models/LED_Init_1')
'''

In [ ]:
print(led_tokenizer)

PreTrainedTokenizerFast(name_or_path='allenai/led-base-16384', vocab_size=50265, model_max_len=16384, is_fast=True, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['[prob_statement]', '[input_statement]', '[output_statement]', '[sample_example]', '[problem_note]', '[prog_language]', '[user_rating]'

In [ ]:
tokenized_texts = led_tokenizer(sentences,padding=True,truncation=True,return_tensors='pt')

In [ ]:
input_ids = tokenized_texts['input_ids']
input_attn = tokenized_texts['attention_mask']

# create 0 global_attention_mask lists
global_attention_mask = len(input_ids) * [[0 for _ in range(len(input_ids[0]))]]

# put global attention on <s> token
global_attention_mask[:, 0] = 1

In [ ]:
print(type(input_attn))

global_attention_mask_pt = torch.tensor(global_attention_mask)

print(len(global_attention_mask_pt),len(global_attention_mask_pt[-100]))
print(global_attention_mask_pt[0])

print(type(global_attention_mask_pt))

In [ ]:
'''
import pickle

with open(project_path+'Pickles/pickle_tokenized_input_led_1.pickle', 'wb') as f:
  pickle.dump((input_ids,input_attn,global_attention_mask_pt),f)
'''

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_tokenized_input_led_1.pickle', 'rb') as f:
  input_ids,input_attn,global_attention_mask = pickle.load(f)

In [ ]:
print(len(input_ids),len(input_ids[-100]))
print(input_ids[0])

tr,val = input_ids[:20000],input_ids[20000:]

print(tr[0])
print(len(tr),len(tr[0]))
print(type(tr))

36565 1472
tensor([    0, 50265,   250,  ...,     1,     1,     1])
tensor([    0, 50265,   250,  ...,     1,     1,     1])
20000 1472
<class 'torch.Tensor'>


In [ ]:
print(led)

In [ ]:
output_ids= []
output_attn = []

for i in range(len(code_data_all)):
  tokenized_code_temp = led_tokenizer(code_data_all[i],padding='max_length',truncation=True, max_length=4096,return_tensors='pt')
  output_ids.append(tokenized_code_temp['input_ids'][0])
  output_attn.append(tokenized_code_temp['attention_mask'][0])


#print(len(tokenized_code['input_ids'][0]),len(tokenized_code['input_ids'][0]))
#print(tokenized_code['attention_mask'][0])

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_tokenized_output_led_1_1.pickle', 'wb') as f:
  pickle.dump((output_ids,output_attn),f)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_tokenized_output_led_1_1.pickle', 'rb') as f:
  output_ids,output_attn = pickle.load(f)

In [ ]:
def tensorify(lst):
    """
    List must be nested list of tensors (with no varying lengths within a dimension).
    Nested list of nested lengths [D1, D2, ... DN] -> tensor([D1, D2, ..., DN)

    :return: nested list D
    """
    # base case, if the current list is not nested anymore, make it into tensor
    if type(lst[0]) != list:
        if type(lst) == torch.Tensor:
            return lst
        elif type(lst[0]) == torch.Tensor:
            return torch.stack(lst, dim=0)
        else:  # if the elements of lst are floats or something like that
            return torch.tensor(lst)
    current_dimension_i = len(lst)
    for d_i in range(current_dimension_i):
        tensor = tensorify(lst[d_i])
        lst[d_i] = tensor
    # end of loop lst[d_i] = tensor([D_i, ... D_0])
    tensor_lst = torch.stack(lst, dim=0)
    return tensor_lst

In [ ]:
output_ids = tensorify(output_ids)

In [ ]:
print(type(output_ids[0]))

<class 'torch.Tensor'>


In [ ]:
output_ids_padding = torch.where(output_ids== led_tokenizer.pad_token_id,-100,output_ids)

In [ ]:
import pickle

output_ids_padding = []

for i,labels in enumerate(output_ids[8700:],8701):
  if(i%100==0):
    print(i)
  temp=[-100 if token == led_tokenizer.pad_token_id else token for token in labels]
  output_ids_padding.append(temp)

  if(i%100==0 or i==len(output_ids)):
    with open(project_path+'Pickles/Temp/pickle_tokenized_output_led_'+str(i)+'.pickle', 'wb') as f:
      pickle.dump(output_ids_padding,f)
    print('Saved: '+'pickle_tokenized_output_led_'+str(i)+'.pickle')

    output_ids_padding = []

8800


In [ ]:
output_ids_padding=[]

for i in range(len(output_ids)):
  i=i+100
  with open(project_path+'Pickles/Temp/pickle_tokenized_output_led_'+str(i)+'.pickle', 'rb') as f:
      op_temp = pickle.load(f)
  for op in op_temp:
    output_ids_padding.append(op_temp)

with open(project_path+'Pickles/Temp/pickle_tokenized_output_led_'+str(len(output_ids))+'.pickle', 'rb') as f:
    op_temp = pickle.load(f)
for op in op_temp:
  output_ids_padding.append(op_temp)


In [ ]:
print(len(output_ids_padding),len(output_ids_padding[-100]))
print(output_ids_padding[0])
print(output_ids[0])

36565 4096
tensor([    0, 50272, 28819,  ...,  -100,  -100,  -100])
tensor([    0, 50272, 28819,  ...,     1,     1,     1])


In [ ]:
print(type(output_attn))

#output_ids_pt = torch.tensor(output_ids_padding)
output_attn_pt = tensorify(output_attn)

print(len(output_ids_padding),len(output_ids_padding[-100]))
print(output_attn_pt[0])

print(type(output_ids_padding))

<class 'list'>
36565 4096
tensor([1, 1, 1,  ..., 0, 0, 0])
<class 'torch.Tensor'>


In [ ]:
import pickle

with open(project_path+'Pickles/pickle_tokenized_output_led_1.pickle', 'wb') as f:
  pickle.dump((output_ids_padding,output_attn_pt),f)

In [ ]:
import pickle

with open(project_path+'Pickles/pickle_tokenized_output_led_1.pickle', 'rb') as f:
  output_ids_,output_attn_ = pickle.load(f)

In [ ]:
print(type(output_ids_))

#output_ids_pt = torch.tensor(output_ids_padding)
#output_attn_pt = tensorify(output_attn)

print(len(output_ids_),len(output_ids_[-100]))
print(output_attn_[0])

print(type(output_attn_))

<class 'torch.Tensor'>
36565 4096
tensor([1, 1, 1,  ..., 0, 0, 0])
<class 'torch.Tensor'>


In [ ]:
import pickle

with open(project_path+'Pickles/pickle_tokenized_input_led_1.pickle', 'rb') as f:
  input_ids,input_attn,global_attention_mask = pickle.load(f)


import pickle
with open(project_path+'Pickles/pickle_tokenized_output_led_1.pickle', 'rb') as f:
  output_ids_,output_attn_ = pickle.load(f)


input_ids_new = []
input_attn_new = []
global_attention_mask_new = []
output_ids_new = []
output_attn_new = []


cnt_1024 = 0
cnt_4096 = 0

for i in range(len(output_ids_)):
  try:
    if(output_ids_[i].tolist().index(-100)+1<=1023):
      cnt_1024=cnt_1024+1

      input_ids_new.append(torch.tensor(input_ids[i]))
      input_attn_new.append(torch.tensor(input_attn[i]))
      global_attention_mask_new.append(torch.tensor(global_attention_mask[i]))
      output_ids_new.append(torch.tensor(output_ids_[i][:1024]))
      output_attn_new.append(torch.tensor(output_attn_[i][:1024]))
    else:
      cnt_4096 = cnt_4096+1
  except:
    cnt_4096 = cnt_4096+1

print(cnt_1024)
print(cnt_4096)


input_ids_new = tensorify(input_ids_new)
input_attn_new = tensorify(input_attn_new)
global_attention_mask_new = tensorify(global_attention_mask_new)
output_ids_new = tensorify(output_ids_new)
output_attn_new = tensorify(output_attn_new)


import pickle

with open(project_path+'Pickles/pickle_tokenized_input_max_ouput_1024_led_1.pickle', 'wb') as f:
  pickle.dump((input_ids_new,input_attn_new,global_attention_mask_new),f)

import pickle

with open(project_path+'Pickles/pickle_tokenized_output_max_ouput_1024_led_1.pickle', 'wb') as f:
  pickle.dump((output_ids_new,output_attn_new),f)

In [ ]:
print(type(input_ids))

<class 'torch.Tensor'>


In [ ]:
led.set_input_embeddings= None

In [ ]:
tokenized_texts = tokenizer(sentences, padding=True)

In [ ]:
print(tokenizer.decode(tokenized_texts['input_ids'][500]))

[CLS] [prob_statement] вконтакте открыла второи штаб в санкт - петербурге! вы не преминули возможностью сменить обстановку и решили переехать из офиса в доме зингера в офис на красном мосту. для комфортнои работы вам потребуются два монитора с одинаковои высотои, чтобы изображение на них выглядело единым целым. на складе офиса на красном мосту есть $ $ $ n $ $ $ мониторов, $ $ $ i $ $ $ - и из них имеет ширину $ $ $ w _ i $ $ $ и высоту $ $ $ h _ i $ $ $. любои монитор можно повернуть на 90 градусов, и тогда он будет иметь ширину $ $ $ h _ i $ $ $ и высоту $ $ $ w _ i $ $ $. назовем неупорядоченную пару из двух различных мониторов подходящеи, если можно их повернуть так, чтобы они имели одинаковую высоту. любои из мониторов в паре можно как повернуть относительно исходнои ориентации, так и не поворачивать. подсчитаите подходящие пары мониторов. [input_statement] входные данныев первои строке задано одно целое число $ $ $ n $ $ $ — число мониторов на складе. в каждои из следующих $ $ $ 

In [ ]:
print(str(df['problem_statement'][40000]))

Lord Omkar has permitted you to enter the Holy Church of Omkar! To test your worthiness, Omkar gives you a password which you must interpret!A password is an array $$$a$$$ of $$$n$$$ positive integers. You apply the following operation to the array: pick any two adjacent numbers that are not equal to each other and replace them with their sum. Formally, choose an index $$$i$$$ such that $$$1 \leq i < n$$$ and $$$a_{i} \neq a_{i+1}$$$, delete both $$$a_i$$$ and $$$a_{i+1}$$$ from the array and put $$$a_{i}+a_{i+1}$$$ in their place. For example, for array $$$[7, 4, 3, 7]$$$ you can choose $$$i = 2$$$ and the array will become $$$[7, 4+3, 7] = [7, 7, 7]$$$. Note that in this array you can't apply this operation anymore.Notice that one operation will decrease the size of the password by $$$1$$$. What is the shortest possible length of the password after some number (possibly $$$0$$$) of operations?


In [ ]:
from transformers import RobertaTokenizer,LongformerTokenizer

tokenizer_code = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

print(len(tokenizer_code))

special_tokens_dict = {'additional_special_tokens': ['[problem_tags]','[problem_difficulty]','[req_time]','[req_memory]','[code_text]']}
num_added_toks = tokenizer_code.add_special_tokens(special_tokens_dict)
#model.resize_token_embeddings(len(tokenizer))

print(len(tokenizer_code))


50265
50270


In [ ]:
code_data = []

for i in range(len(df['problem_id'])):
  #print(i)
  data = ''
  data = '[problem_tags]' + ('' if df['problem_tags'][i] is np.NaN else str(df['problem_tags'][i]))
  data = data + '[problem_difficulty]'+('' if df['problem_difficulty'][i] is np.NaN else str(df['problem_difficulty'][i]))
  data = data +'[req_time]'+('' if df['req_time'][i] is np.NaN else str(df['req_time'][i]))
  data = data +'[req_memory]'+('' if df['req_memory'][i] is np.NaN else str(df['req_memory'][i]))
  data = data +'[code_text]'+('' if df['submitted_code'][i] is np.NaN else str(df['submitted_code'][i]))
  code_data.append(data)

In [ ]:
print(len(code_data))

55347


In [ ]:
tokenized_codes = tokenizer_code(code_data_all[25000:30000], padding=True)

In [ ]:
print(len(tokenized_codes['input_ids'][0]))

34334


In [ ]:
print(tokenized_codes['input_ids'][0])

[0, 50265, 8007, 12228, 50266, 1570, 612, 50267, 541, 50141, 4339, 50268, 2619, 50141, 36640, 50269, 10431, 47209, 41552, 438, 47717, 1020, 15698, 50121, 50121, 50118, 10431, 47209, 41552, 337, 47549, 15698, 50121, 50121, 50118, 10431, 47209, 41552, 438, 20951, 15698, 50121, 50121, 50118, 10928, 49010, 49008, 131, 50121, 50121, 50118, 2544, 295, 6, 102, 10975, 47303, 245, 44082, 50121, 50121, 50118, 2544, 1049, 43048, 50121, 50121, 50118, 45152, 50121, 50121, 50118, 50117, 43511, 506, 46469, 207, 417, 1297, 947, 282, 4397, 50121, 50121, 50118, 50117, 1990, 1640, 2544, 939, 5214, 134, 131, 118, 41552, 282, 131, 118, 49346, 50121, 50121, 50118, 50117, 45152, 50121, 50121, 50118, 50117, 50117, 49775, 46469, 116, 7606, 417, 7606, 417, 37457, 282, 1297, 118, 6, 282, 4397, 50121, 50121, 50118, 50117, 50117, 3145, 462, 3810, 1640, 47717, 995, 4397, 50121, 50121, 50118, 50117, 50117, 43511, 506, 46469, 207, 417, 1297, 947, 102, 10975, 118, 48601, 50121, 50121, 50118, 50117, 24303, 50121, 50121

In [ ]:
print(tokenizer_code.decode(tokenized_codes['input_ids'][0]))



























}</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
print(tokenizer_code.convert_ids_to_tokens(tokenized_codes['input_ids'][0]))

['<s>', '[problem_tags]', 'inter', 'active', '[problem_difficulty]', '14', '00', '[req_time]', '30', 'Âł', 'ms', '[req_memory]', '200', 'Âł', 'KB', '[code_text]', '#', 'include', '<', 'c', 'std', 'io', '>', 'č', 'č', 'Ċ', '#', 'include', '<', 'al', 'gorithm', '>', 'č', 'č', 'Ċ', '#', 'include', '<', 'c', 'string', '>', 'č', 'č', 'Ċ', 'using', 'Ġnamespace', 'Ġstd', ';', 'č', 'č', 'Ċ', 'int', 'Ġn', ',', 'a', '[', '10000', '5', '];', 'č', 'č', 'Ċ', 'int', 'Ġmain', '()', 'č', 'č', 'Ċ', '{', 'č', 'č', 'Ċ', 'ĉ', 'scan', 'f', '("', '%', 'd', '",', '&', 'n', ');', 'č', 'č', 'Ċ', 'ĉ', 'for', '(', 'int', 'Ġi', '=', '1', ';', 'i', '<', 'n', ';', 'i', '++)', 'č', 'č', 'Ċ', 'ĉ', '{', 'č', 'č', 'Ċ', 'ĉ', 'ĉ', 'printf', '("', '?', 'Ġ%', 'd', 'Ġ%', 'd', '\\', 'n', '",', 'i', ',', 'n', ');', 'č', 'č', 'Ċ', 'ĉ', 'ĉ', 'ff', 'l', 'ush', '(', 'std', 'out', ');', 'č', 'č', 'Ċ', 'ĉ', 'ĉ', 'scan', 'f', '("', '%', 'd', '",', '&', 'a', '[', 'i', ']);', 'č', 'č', 'Ċ', 'ĉ', '}', 'č', 'č', 'Ċ', 'ĉ', 'for', '(', 'i

In [ ]:
unk_count=0

print(tokenizer_code.unk_token,tokenizer_code.unk_token_id)

for i in range(len(tokenized_codes['input_ids'])):
  if(tokenizer_code.unk_token_id in tokenized_codes['input_ids'][i]):
    unk_count = unk_count+1

print(unk_count)

<unk> 3
0


In [ ]:
max_length = 0
max_ind = -1

for i in range(len(tokenized_codes['input_ids'])):
  if(len(tokenizer_code.decode(tokenized_codes['input_ids'][i]))>=max_length):
    max_length = len(tokenizer_code.decode(tokenized_codes['input_ids'][i]))
    max_ind = i

print(tokenizer_code.decode(tokenized_codes['input_ids'][max_ind]))
print(max_length,max_ind)


</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
max_length = 0
max_ind = -1

for i in range(len(code_data_all)):
  if(len(code_data_all[i])>=max_length):
    max_length = len(code_data_all[i])
    max_ind = i

print(code_data_all[max_ind])
sss = tokenizer_code(code_data_all[max_ind])
print(max_length,len(sss['input_ids']))
print(max_ind)

In [ ]:
ss = 0
for i in range(len(tokenized_codes['input_ids'])):
  ss=ss+len(tokenized_codes['input_ids'][i])

print(ss/len(tokenized_codes['input_ids']))

1600.3688


In [ ]:
print(str(df['user_rating'][20000]))
print(df['submitted_code'][20])

In [ ]:
import re

comment_remover = '(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)'

temp = 'char *note = "You can make a one-line comment with //";'

ccc = re.sub(comment_remover, '', temp)

print(temp)
print('-------')
print(ccc)

char *note = "You can make a one-line comment with //";
-------
char *note = "You can make a one-line comment with 


In [ ]:
import re

temp = df['problem_statement'][45002]

print(temp)

ppp = re.sub('\$','',temp)
ppp = re.sub(r'\\[a-z]+','',ppp)
ppp = re.sub(r'\\','',ppp)

print(ppp)

Polygon is not only the best platform for developing problems but also a square matrix with side $$$n$$$, initially filled with the character 0.On the polygon, military training was held. The soldiers placed a cannon above each cell in the first row and a cannon to the left of each cell in the first column. Thus, exactly $$$2n$$$ cannons were placed.  Initial polygon for $$$n=4$$$. Cannons shoot character 1. At any moment of time, no more than one cannon is shooting. When a 1 flies out of a cannon, it flies forward (in the direction of the shot) until it collides with a polygon border or another 1. After that, it takes the cell in which it was before the collision and remains there. Take a look at the examples for better understanding.More formally:   if a cannon stands in the row $$$i$$$, to the left of the first column, and shoots with a 1, then the 1 starts its flight from the cell ($$$i, 1$$$) and ends in some cell ($$$i, j$$$);  if a cannon stands in the column $$$j$$$, above the 

In [ ]:
from transformers import EncoderDecoderModel

In [ ]:
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relatio

In [ ]:
codebert2codebert = EncoderDecoderModel.from_encoder_decoder_pretrained("microsoft/codebert-base","microsoft/codebert-base")

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['encoder.layer.11.crossattention.self.value.bias', 'encoder.layer.6.crossattention.self.query.bias', 'encoder.layer.10.crossattention.self.query.weight', 'encoder.layer.6.crossattention.self.query.weight', 'encoder.layer.1.crossattention.self.query.weight', 'encoder.layer.4.crossattention.self.value.weight', 'lm_head.dense.weight', 'encoder.layer.2.crossattention.output.dense.weight', 'encoder.layer.11.crossattention.self.query.weight', 'encoder.layer.1.crossattention.output.dense.weight', 'encoder.layer.7.crossattention.self.query.bias', 'encoder.layer.3.crossattention.output.LayerNorm.bias', 'encoder.layer.11.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.output.LayerNorm.weight', 'encoder.layer.5.crossattention.output.dense.bias', 'encoder.layer.11.crossattention.self.key.weight', 'encoder.layer.0.crossattention.self.value

In [ ]:
from transformers import LongformerModel, LongformerTokenizer, BertTokenizer
#lf = LongformerModel.from_pretrained("microsoft/codebert-base")
lt = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096",)

print(lt.vocab_size)

50265


In [ ]:
from transformers import LongformerModel, LongformerTokenizer
lt = LongformerTokenizer.from_pretrained("microsoft/codebert-base")

etl = lt('hi this is me', max_length=4097, truncation=True, padding='max_length', return_tensors="pt")
#print(ltb.decode(et['input_ids']))
print(len(etl['input_ids'][0]))
print(lt.decode(etl['input_ids'][0]))

4097
<s>hi this is me</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
from transformers import LongformerForSequenceClassification

lf_classifier = LongformerForSequenceClassification.from_pretrained("microsoft/codebert-base")
logits = lf_classifier(**etl).logits
print(logits)


You are using a model of type roberta to instantiate a model of type longformer. This is not supported for all configurations of models and can yield errors.
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['encoder.layer.9.attention.self.query_global.bias', 'encoder.layer.5.attention.self.key_global.bias', 'encoder.layer.3.attention.self.key_global.bias', 'encoder.layer.4.attention.self.query_global.weight', 'encoder.layer.6.attention.self.value_global.weight', 'encoder.layer.1.attention.self.key_global.weight', 'encoder.layer.5.attention.self.value_global.weight', 'encoder.layer.11.attention.self.key_global.weight', 'encoder.layer.3.attention.self.key_global.weight', 'classifier.dense.bias', 'encoder.layer.8.attention.self.value_global.bias', 'encoder.layer.3.attention.self.query_global.bias', 'encoder.layer.10.attention.self.key_global.bias', 'encoder.layer.8.attention.self.query

tensor([[0.2389, 0.1842]], grad_fn=<AddmmBackward>)


In [ ]:
from transformers import BertTokenizer

ltb = BertTokenizer.from_pretrained("bert-base-uncased")

et = ltb('hi this is me', max_length=513, truncation=True, padding='max_length', return_tensors="pt")
#print(ltb.decode(et['input_ids']))
print(len(et['input_ids'][0]))
print(ltb.convert_ids_to_tokens(et['input_ids'][0]))

513
['[CLS]', 'hi', 'this', 'is', 'me', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [ ]:
from transformers import BertForSequenceClassification
bertModel = BertForSequenceClassification.from_pretrained("bert-base-uncased")
logits = bertModel(**et).logits
print(logits)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: ignored

In [ ]:
longformer2codebert = EncoderDecoderModel.from_encoder_decoder_pretrained("allenai/longformer-base-4096","microsoft/codebert-base")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['encoder.layer.5.crossattention.self.query.bias', 'lm_head.dense.bias', 'encoder.layer.6.crossattention.output.dense.bias', 'encoder.layer.8.crossattention.output.dense.weight', 'encoder.layer.2.crossattention.self.query.weight', 'encoder.layer.4.crossattention.output.dense.weight', 'encoder.layer.9.crossattention.self.query.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.6.crossattention.self.key.weight', 'encoder.layer.7.crossattention.self.value.bias', 'lm_head.dense.weight', 'encoder.layer.11.crossattention.output.dense.bias', 'encoder.layer.11.crossattention.self.key.weight', 'encoder.layer.4.crossattention.self.query.weight', 'encoder.layer.4.crossattention.self.value.weight', 'encoder.layer.3.crossattention.self.query.weight', 'encoder.layer.1.crossattention.output.dense

In [ ]:
from transformers import T5Tokenizer
from transformers import BertLMHeadModel

#tokenizer = T5Tokenizer.from_pretrained('t5-small')
bertLM = BertLMHeadModel.from_pretrained("bert-base-uncased")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bertLM.save_pretrained('./bertLM')

In [ ]:
bertLM2bertLM = EncoderDecoderModel.from_encoder_decoder_pretrained("./bertLM","./bertLM")

Some weights of the model checkpoint at ./bertLM were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./bertLM and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN

In [ ]:
decoder_config = bertLM2bertLM.config.decoder

In [ ]:
print(decoder_config.add_cross_attention)

True


In [ ]:
from transformers import RobertaForSequenceClassification

rf_classifier = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base")
#logits = lf_classifier(**etl).logits
#print(logits)


Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be 

In [ ]:
project_path='/content/drive/My Drive/Machine Learning/My HuggingFace/Encoder Decoder/'

In [ ]:
cd $project_path

/content/drive/My Drive/Machine Learning/My HuggingFace/Encoder Decoder


In [ ]:
ls

Bert_FR_1.ipynb     dump.sql         mysql2sqlite/
codeforcesdump.sql  ED_test_1.ipynb  out.json


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
!pip install mysql

In [ ]:
conn = sqlite3.connect(project_path+'test.db')
print("Opened database successfully");

Opened database successfully


In [ ]:
connection = sqlite3.connect(":memory:")
#Create the database in RAM

cursor = connection.cursor()

sql_file = open("codeforcesdump.sql")
sql_as_string = sql_file.read()
cursor.executescript(sql_as_string,)

OperationalError: ignored

In [ ]:
c = sqlite3.connect(":memory:")

# Open and read the file as a single buffer
fd = open('codeforcesdump.sql', 'r')
sqlFile = fd.read()
fd.close()

# all SQL commands (split on ';')
sqlCommands = sqlFile.split(';')

# Execute every command from the input file
for command in sqlCommands:
    # This will skip and report errors
    # For example, if the tables do not yet exist, this will skip over
    # the DROP TABLE commands
    try:
        c.execute(command)
    except:
        print ("Command skipped: ")

In [ ]:
for row in cursor.execute("SELECT * FROM airports"):
    print(row)
conn.close()

In [ ]:
df = pd.read_json('out.json',lines=True,chunksize=20)

In [ ]:
df2= pd.DataFrame(df)
df2.head()

ValueError: ignored

In [ ]:
with open('out.json', 'rb') as f:
    data = f.read()
    print(data)

In [ ]:
! git clone https://github.com/dumblob/mysql2sqlite.git

Cloning into 'mysql2sqlite'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 137 (delta 4), reused 6 (delta 2), pack-reused 126
Receiving objects: 100% (137/137), 43.13 KiB | 2.16 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [ ]:
cd mysql2sqlite

/content/drive/My Drive/Machine Learning/My HuggingFace/Encoder Decoder/mysql2sqlite


In [ ]:
mysql2sqlite/mysql2sqlite codeforcesdump.sql | sqlite3 mysqlite3.db

SyntaxError: ignored

In [ ]:
!pip install pymysql

     |████████████████████████████████| 51kB 6.5MB/s 


In [ ]:
%load_ext sql

In [ ]:
import pymysql
conn = pymysql.connect(db='mysql')
cur = conn.cursor()

OperationalError: ignored

In [ ]:
#!pip install simpletransformers

In [ ]:
import numpy as np
import pandas as pd

dataset = pd.read_csv(project_path+'fake_or_real_news.csv')
print(dataset.shape)

texts=[]
texts=dataset['text']#####################################
label=dataset['label']

from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))


#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

training_size=int(0.8*dataset.shape[0])
print(dataset.shape[0],training_size)
data_train=dataset[:training_size]['text']
y_train_val=y[:training_size]
data_test=dataset[training_size:]['text']
y_test=y[training_size:]


(6335, 4)
6335 5068


In [ ]:
sentences = data_train
print(sentences)
labels = y_train_val[:,0]
#labels = y_train
print(labels)

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
5063    By Nick Bernabe \nFollowing the recent mass ar...
5064    10 Comments on "What Does Washington ‘Plan B’ ...
5065    Less than six weeks on his powerful Capitol Hi...
5066    $23 Russell Brand’s Back on ‘The Trews’ and He...
5067    UFO over the East Valley here in AZ last night...
Name: text, Length: 5068, dtype: object
[0 0 1 ... 1 0 0]


In [ ]:
print('Max sentence length: ', max([len(sen.split(' ')) for sen in sentences]))

Max sentence length:  20897


In [ ]:
#sentences=sentences[:60000]
#labels=labels[:60000]

In [ ]:
print(sentences[:1])

0    Daniel Greenfield, a Shillman Journalism Fello...
Name: text, dtype: object


In [ ]:
TRAIN_SPLIT = 0.8

nb_train_samples = int(TRAIN_SPLIT * sentences.shape[0])

x_train = np.array(sentences[:nb_train_samples])
y_train = np.array(labels[:nb_train_samples])
x_val = np.array(sentences[nb_train_samples:])
y_val = np.array(labels[nb_train_samples:])

#print(x_train[0])
#print(x_val[0])

In [ ]:
import pandas as pd
ll=[]
for i in range(len(x_train)):
  ll.append([x_train[i],y_train[i]])

train_df = pd.DataFrame(ll,columns=['text','labels'])


ll=[]
for i in range(len(x_val)):
  ll.append([x_val[i],y_val[i]])

val_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
sentences_test = np.array(data_test)
print(sentences_test[0])
labels_test = y_test[:, 0]
#labels = y_train
print(labels_test)

Complaining about Western “hysteria” surrounding repeated predictions of Russian military attacks on NATO member nations, Russian President Vladimir Putin sought to resolve two solid years of predictions to that effect with a straightforward assurance that “Russia is not going to attack anyone.” 
Via AntiMedia 

Putin accused Western nations of having “mythical, dreamt-up problems,” and insisted the idea that Russia was going to attack the West was “simply stupid and unrealistic.” He added that he believes the idea is being played up to justify bigger military spending.
The Russian president also sought to dismiss allegations that he is plotting to rig the US elections to his own benefit, noting that the US is a great power and not some banana republic with an easily manipulated political system. Russia has repeatedly denied involvement in such plots.
NATO has played up the Russian threat to justify sending over 40,000 ground troops to the Russian border, with ever-growing numbers anno

In [ ]:
ll=[]
for i in range(len(sentences_test)):
  ll.append([sentences_test[i],labels_test[i]])

test_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
print('Training sample: ' + str(len(train_df['text'])))
print('Validation sample: ' + str(len(val_df['text'])))
print('Testing sample: ' + str(len(test_df['text'])))

Training sample: 4054
Validation sample: 1014
Testing sample: 1267


In [ ]:
model_type = "bert"
model_name = "bert-base-cased"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": "/content/drive/My Drive/Machine Learning/Fake News/Evaluation/Evaluation/Transformer/output/"+model_type,
    "best_model_dir": "/content/drive/My Drive/Machine Learning/Fake News/Evaluation/Evaluation/Transformer/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 64,
    "wandb_project": "Fake News FR_1 Bert",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#import pandas as pd

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 1
train_args["evaluate_during_training"] = True
train_args["num_train_epochs"] = 10
train_args["use_early_stopping"] = True


# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, args=train_args)

# Train the model
model.train_model(train_df,eval_df = val_df)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

wandb: Currently logged in as: tawkat (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb/run-20200918_200733-6tg3uaj7
wandb: Syncing run bert-base-cased


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

wandb: Waiting for W&B process to finish, PID 1266
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200918_113703-15wcuhoi/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200918_113703-15wcuhoi/logs/debug-internal.log
wandb: Run summary:
wandb:                                                       Training loss 5.582822996075265e-05
wandb:                                                                  lr 2.1646202819463395e-05
wandb:                                                         global_step 1150
wandb:                                                               _step 22
wandb:                                                            _runtime 3593
wandb:                                                          _timestamp 1600432617
wandb: Run history:
wandb:   Training loss █▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:              lr ▁▅███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃
wandb:     global_step ▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
wandb:           _step

In [ ]:
predictions, raw_outputs = model.predict(test_df['text'])

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_df['labels'],predictions))

0.9629044988161011


In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Bert_1_FR.pickle', 'wb') as f:
  pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Bert_1_FR.pickle', 'rb') as f:
  y_true, y_pred = pickle.load(f)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.9629044988161011
